# Examples of Advanced queries with roles

In [1]:
import pymongo

In [2]:
db = pymongo.MongoClient()['eventnet']['roles']

## Query with role

Find `SPEECH` where `SPEAKER`

In [49]:
event = 'SPEECH'
role = 'SPEAKER'

query = {
    'events': {
        '$elemMatch': {
            'label': event,
            'roles': {
                '$elemMatch': {'label': role}
                }
            }
        }
    }

data = list(db.find(query))
for e_data in data[:4]:
    print("Document {}".format(e_data['id']))
    print("----------------")
    for e in e_data['events']:
        if e['label'] == event:
            print(e['label'], e['text'])
            for r in e['roles']:
                print('\t', r['label'], r['text'])
    print("=================")

Document CS_19480215_art0000010
----------------
SPEECH discorso
	 SPEAKER lui suo
	 TIME del marzo di settembre
	 SPEAKER di Stalin
	 TIME del marzo
Document CS_19480229_art0000012
----------------
SPEECH None
	 SPEAKER il segretario generale sindacale Sailliint
Document CS_19480212_art0000013
----------------
SPEECH discorso
	 SPEAKER del ministro degli Esteri svedese
Document CS_19480220_art0000008
----------------
SPEECH il discorso pronuncia
	 TIME ieri sera
	 PLACE a Louisville
	 SPEAKER da uno dei più alti funzionari del Dipartimento di Stato
	 SPEAKER suo
	 SPEAKER da Truman
	 TIME stasera di stasera
	 VENUE al tradizionale banchetto per il Jackson anda Jefferson Day »
	 TIME del mese scorso


## Query with role and value

Find `SPEECH` where `SPEAKER` is "Truman"

In [50]:
event = 'SPEECH'
role = 'SPEAKER'
value = 'truman'


query = {
    'events': {
        '$elemMatch': {
            'label': event,
            'roles': {
                '$elemMatch': {
                    'label': role,
                    'text': {'$regex': '(?i){}'.format(value)}
                    }
                }
            }
        }
    }

data = list(db.find(query))
for e_data in data[:4]:
    print("Document {}".format(e_data['id']))
    print("----------------")
    for e in e_data['events']:
        if e['label'] == event:
            print(e['label'], e['text'])
            for r in e['roles']:
                print('\t', r['label'], r['text'])
    print("=================")

Document CS_19480220_art0000008
----------------
SPEECH il discorso pronuncia
	 TIME ieri sera
	 PLACE a Louisville
	 SPEAKER da uno dei più alti funzionari del Dipartimento di Stato
	 SPEAKER suo
	 SPEAKER da Truman
	 TIME stasera di stasera
	 VENUE al tradizionale banchetto per il Jackson anda Jefferson Day »
	 TIME del mese scorso
Document CS_19480319_art0000004
----------------
SPEECH discorso
	 SPEAKER di Truman
Document CS_19480320_art0000005
----------------
SPEECH discorso
	 SPEAKER suo
	 SPEAKER suo
	 SPEAKER di Truman
	 SPEAKER , segretario del
Document CS_19480320_art0000010
----------------
SPEECH discorso
	 SPEAKER di Truman


## Query with any role and value

Find `SPEECH` where `ANY` is "stati uniti"

In [52]:
event = 'SPEECH'
value = 'stati uniti'

query = {
    'events': {
        '$elemMatch': {
            'label': event,
            'roles': {
                '$elemMatch': {
                    'text': {'$regex': '(?i){}'.format(value)}
                    }
                }
            }
        }
    }

data = list(db.find(query))
for e_data in data[:10]:
    print("Document {}".format(e_data['id']))
    print("----------------")
    for e in e_data['events']:
        if e['label'] == event:
            print(e['label'], e['text'])
            for r in e['roles']:
                print('\t', r['label'], r['text'])
    print("=================")

Document CS_19480210_art0000038
----------------
SPEECH pronunciare un discorso discorso
	 TIME Quella stessa sera
	 SPEAKER Roosevelt
	 VENUE radio
	 TOPIC sulla politica degli i Stati Uniti
Document CS_19480319_art0000003
----------------
SPEECH discorso pronunciato
	 TIME Ieri
	 SPEAKER dal Presidente degli Stati Uniti
Document CS_19480402_art0000002
----------------
SPEECH un discorso pronunciato
	 PLACE a Madison , nel Wisconsin
	 SPEAKER ambasciatore norvegese negli Stati Uniti barone Morgenstierne
Document CS_19481026_art0000005
----------------
SPEECH discorso
	 SPEAKER del delegato degli Stati Uniti
	 CONTAINING_EVENT questa sessione


## Available roles
Get the available roles given an event type label

E.g., get all the possible roles for the event `SPEECH`

In [53]:
event = 'SPEECH'
pipeline = [
    {
        '$unwind': {'path': "$events"}
    }, 
    {
        '$project': {'events': 1, '_id': 0}
    },
    {
        '$match': {"events.label": "{}".format(event)}
    },
    {
        '$unwind': {'path': "$events.roles"}
    },
    {
        '$group': {'_id': "$events.roles.label", 'count': {'$sum': 1}}
    },
    {
        '$sort': {'count': -1}
    }
]
roles = list(db.aggregate(pipeline))
for record in roles[:10]:
    print(record)

{'_id': 'SPEAKER', 'count': 1017}
{'_id': 'TIME', 'count': 279}
{'_id': 'PLACE', 'count': 152}
{'_id': 'TOPIC', 'count': 130}
{'_id': 'VENUE', 'count': 122}
{'_id': 'AUDIENCE', 'count': 60}
{'_id': 'CONTAINING_EVENT', 'count': 21}
{'_id': 'CONTENT', 'count': 19}
{'_id': 'ITERATIONS', 'count': 14}
{'_id': 'PRECEDING_EVENT', 'count': 11}
